In [52]:
import pandas as pd
from datetime import datetime
from slugify import slugify

In [53]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [54]:
now = datetime.now()
dt_string = now.strftime("%Y-%m-%d %H:%M:%S")

In [55]:
df = pd.read_csv('csv/products.csv')
priceDf = pd.read_csv('out/price.csv')
usersDf = pd.read_csv('csv/users.csv')
users = pd.read_csv('out/users.csv')
locDf = pd.read_csv('out/localities.csv')
prodDf = pd.read_csv('out/products.csv')

In [56]:
df1 = df[['Price 1', 'Curr1', 'Number Review 1', 'Review 1', 'Rating 1', 'ID', 'Name', 'Curr1 ID']]
df2 = df[['Price 2', 'Curr2', 'Number Review 2', 'Review 2', 'Rating 2', 'ID', 'Name', 'Curr2 ID']]
df3 = df[['Price 3', 'Curr3', 'Number Review 3', 'Review 3', 'Rating 3', 'ID', 'Name', 'Curr3 ID']]
df4 = df[['Price 4', 'Curr4', 'Number Review 4', 'Review 4', 'Rating 4', 'ID', 'Name', 'Curr4 ID']]
df5 = df[['Price 5', 'Curr5', 'Number Review 5', 'Review 5', 'Rating 5', 'ID', 'Name', 'Curr5 ID']]

In [57]:
df1 = df1.rename(columns={'Price 1' : 'Price', 'Curr1' : 'Curr', 'Number Review 1' : 'Number Review', \
                          'Review 1' : 'Review', 'Rating 1' : 'Rating', 'Curr1 ID' : 'CurID'})
df2 = df2.rename(columns={'Price 2' : 'Price', 'Curr2' : 'Curr', 'Number Review 2' : 'Number Review', \
                          'Review 2' : 'Review', 'Rating 2' : 'Rating', 'Curr2 ID' : 'CurID'})
df3 = df3.rename(columns={'Price 3' : 'Price', 'Curr3' : 'Curr', 'Number Review 3' : 'Number Review', \
                          'Review 3' : 'Review', 'Rating 3' : 'Rating', 'Curr3 ID' : 'CurID'})
df4 = df4.rename(columns={'Price 4' : 'Price', 'Curr4' : 'Curr', 'Number Review 4' : 'Number Review', \
                          'Review 4' : 'Review', 'Rating 4' : 'Rating', 'Curr4 ID' : 'CurID'})
df5 = df5.rename(columns={'Price 5' : 'Price', 'Curr5' : 'Curr', 'Number Review 5' : 'Number Review', \
                          'Review 5' : 'Review', 'Rating 5' : 'Rating', 'Curr5 ID' : 'CurID'})

In [58]:
newDf = pd.concat([df1, df2, df3, df4, df5])

In [59]:
newDf.rename(columns = {'Review':'review', 'Rating' : 'rating'}, inplace = True)

In [60]:
columns = ['id', 'createdDate', 'updatedDate', 'name', 'internalNote', 'isActive', 'authorId', 'statusId', \
          'productId', 'priceId', 'placeId', 'slug']

In [61]:
newDf = pd.concat([newDf, pd.DataFrame(columns = columns)])

In [62]:
import random
import time
 
 
def str_time_prop(start, end, time_format, prop):
    stime = time.mktime(time.strptime(start, time_format))
    etime = time.mktime(time.strptime(end, time_format))
    ptime = stime + prop * (etime - stime)
    return time.strftime(time_format, time.localtime(ptime))
 
 
def random_date(start, end, prop):
    return str_time_prop(start, end, '%Y-%m-%d', prop)

In [63]:
newDf['createdDate'] = [random_date("2021-1-1", "2022-4-30", random.random()) for i in range(1, newDf.shape[0] + 1)]
newDf['updatedDate'] = [i for i in newDf['createdDate']]
newDf['internalNote'] = ''
newDf['isActive'] = True
newDf['statusId'] = 4
newDf['placeId'] = ''

In [64]:
import random

In [65]:
def set_slug(value):
    return slugify('story' + str(value) + str(random.random()))

newDf['slug'] = newDf['ID'].apply(set_slug)

In [66]:
def get_price_id(value):
    try:
        return priceDf[:][priceDf['ID'] == str(value)]['id'].values[0]
    except:
        return ''

newDf['priceId'] = newDf['CurID'].apply(get_price_id)

In [67]:
newDf['ID'] = newDf['ID'].astype(int)

In [68]:
def get_product_id(value):
    try:
        return prodDf[:][prodDf['id'] == int(value)]['id'].values[0]
    except:
        return ''

newDf['productId'] = newDf['ID'].apply(get_product_id)

In [69]:
def get_product_name(value):
    try:
        return prodDf[:][prodDf['id'] == int(value)]['name'].values[0]
    except:
        return ''
    
newDf['name'] = newDf['ID'].apply(get_product_name)

In [70]:
def get_author_id(value):
    try:
        temp = usersDf[:][(usersDf['Review1'] == str(value)) | \
         (usersDf['Review2'] == str(value)) | \
         (usersDf['Review3'] == str(value)) | \
         (usersDf['Review4'] == str(value)) | \
         (usersDf['Review6'] == str(value)) | \
         (usersDf['Review5'] == str(value))]['Username'].values[0].replace('\n', '')
        return users[:][users['username'].astype(str) == str(temp)]['id'].values[0]
    except:
        return ''

newDf['authorId'] = newDf['Number Review'].apply(get_author_id)

In [71]:
newDf = newDf.drop(['Price', 'Curr', 'Number Review', 'ID', 'Name'], axis=1)

In [72]:
newDf['rating'] = newDf['rating'].fillna(0)
newDf['id'] = [i for i in range(1, newDf.shape[0]+1)]

In [73]:
newDf.shape

(9715, 15)

In [74]:
temp4 = newDf[:][newDf['authorId'] == '']

In [76]:
filter = newDf["authorId"] != ""
newDf = newDf[filter]

In [77]:
newDf.to_csv('out/stories.csv', index=False, encoding='utf-8')

In [85]:
newDf.shape

(8870, 15)

In [81]:
newDf[:][newDf['productId'] == 1316]

,review,rating,CurID,id,createdDate,updatedDate,name,internalNote,isActive,authorId,statusId,productId,priceId,placeId,slug
1315,I paid 50 euros for the cooking class. A great...,5.0,1316_1,1316,2022-03-10,2022-03-10,Cooking Class,,True,1567,4,1316,1271,,story1316-00-1869804074649446
1315,Cooking class costs 600 dh per person for a gr...,5.0,1316_2,3259,2021-06-21,2021-06-21,Cooking Class,,True,2126,4,1316,3084,,story1316-00-2422658321032004
1315,We went shopping for the ingredients for the m...,5.0,1316_3,5202,2021-06-05,2021-06-05,Cooking Class,,True,2175,4,1316,4800,,story1316-00-3492077014126911
1315,Hello. I took the cooking class in Fez and pai...,5.0,1316_4,7145,2021-03-08,2021-03-08,Cooking Class,,True,1892,4,1316,6401,,story1316-00-5821402753487351
1315,I paid 600 dh/ £40 per person for the cooking ...,5.0,1316_5,9088,2021-12-08,2021-12-08,Cooking Class,,True,2392,4,1316,7883,,story1316-00-7353865023680596
